### Data Wrangling

In [1]:
# all modules
import warnings
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import seaborn as sns
import numpy as np


Bad key "text.kerning_factor" on line 4 in
/home/carmencuadrado/miniconda3/envs/Ironhack/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# global stuff
warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_rows', 13)
pd.set_option('display.max_columns', 13)

In [3]:
# load dataset
jobs_db = pd.read_csv(r'../data/processed/jobs_db.csv')

In [4]:
api_data=pd.read_json(r'../data/processed/api_data.json')
api_data

,normalized_job_title,parent_uuid,suggestion,uuid
0,data operations director,bb4e5066e69e4228172b4850d2a920f4,Data Operations Director,8fd068c8d9be73abfa678856177b6c40
1,data processing manager,bb4e5066e69e4228172b4850d2a920f4,Data Processing Manager,ac47656fd51c2cd8037057262c910dc4
10,database engineer,596226d5f9ef63de41a852826d483d19,Database Engineer,16b67d7f57698455aefbc1ae1b358b50
100,data conversion operator,78e0562032df97cc82fcc48388cdf34b,Data Conversion Operator,04b0eaf624b2deb45ccd62cf6ec7c7bc
101,data processing control clerk,9b44dbc6767274d8b3a820e133803d00,Data Processing Control Clerk,2616d64becb4f2c478dd42f501bbfbff
102,maintenance data analyst,9b44dbc6767274d8b3a820e133803d00,Maintenance Data Analyst,25fe195cbf915c0824fa6c44e6d0008c
...,...,...,...,...
94,health data analyst,db06efbaa9cb31d42e02c047ffb0a15a,Health Data Analyst,6cf4d336112984368f83181ac5ffe948
95,medical data analyst,db06efbaa9cb31d42e02c047ffb0a15a,Medical Data Analyst,7acaed29536648e45ea2b82d45047fcf
96,data processing services sales representative,96fe8c70b3c2f9ca197722fe88c5d6ca,Data Processing Services Sales Representative,f670246beda8187fbe875401e7bc4ba6


In [5]:
url='https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html=requests.get(url).content
soup = BeautifulSoup(html, 'lxml')
countries = soup.find_all('td')

In [37]:
lista=[]
lista=[i.text for i in countries]
cleaned=[i.strip() for i in lista if len(i)>3]
split=2
country_codes=[cleaned[i:i+split] for i in range(0,len(cleaned),split)]

[['Belgium', '(BE)'],
 ['Greece', '(EL)'],
 ['Lithuania', '(LT)'],
 ['Portugal', '(PT)'],
 ['Bulgaria', '(BG)'],
 ['Spain', '(ES)'],
 ['Luxembourg', '(LU)'],
 ['Romania', '(RO)'],
 ['Czechia', '(CZ)'],
 ['France', '(FR)'],
 ['Hungary', '(HU)'],
 ['Slovenia', '(SI)'],
 ['Denmark', '(DK)'],
 ['Croatia', '(HR)'],
 ['Malta', '(MT)'],
 ['Slovakia', '(SK)'],
 ['Germany', '(DE)'],
 ['Italy', '(IT)'],
 ['Netherlands', '(NL)'],
 ['Finland', '(FI)'],
 ['Estonia', '(EE)'],
 ['Cyprus', '(CY)'],
 ['Austria', '(AT)'],
 ['Sweden', '(SE)'],
 ['Ireland', '(IE)'],
 ['Latvia', '(LV)'],
 ['Poland', '(PL)'],
 ['United Kingdom', '(UK)'],
 ['Iceland', '(IS)'],
 ['Norway', '(NO)'],
 ['Liechtenstein', '(LI)'],
 ['Switzerland', '(CH)'],
 ['Montenegro', '(ME)'],
 ['North Macedonia', '(MK)'],
 ['Albania', '(AL)'],
 ['Serbia', '(RS)'],
 ['Turkey', '(TR)'],
 ['Bosnia and Herzegovina', '(BA)'],
 ['Kosovo*[1]', '(XK[2])'],
 ['Armenia', '(AM)'],
 ['Belarus', '(BY)'],
 ['Georgia', '(GE)'],
 ['Azerbaijan', '(AZ)'],
 ['M

In [7]:
country_codes=pd.DataFrame(country_codes)
country_codes=country_codes.rename(columns={0:'country',1:'country_code'})
country_codes['country_code']=country_codes['country_code'].str.replace(')','')
country_codes['country_code']=country_codes['country_code'].str.replace('(','')
country_codes

,country,country_code
0,Belgium,BE
1,Greece,EL
2,Lithuania,LT
3,Portugal,PT
4,Bulgaria,BG
5,Spain,ES
...,...,...
66,New Zealand,NZ
67,Taiwan,TW
68,Canada,CA


In [8]:
#Rename columnsdf=pd.merge(jobs_df,api_data, left_on='uuid',right_on='uuid')
jobs_db=jobs_db.rename(columns={'uuid':'person_id','dem_education_level':'ed_level','dem_full_time_job':'job_type','question_bbi_2016wave4_basicincome_awareness':'bi_awareness','question_bbi_2016wave4_basicincome_vote':'bi_vote','question_bbi_2016wave4_basicincome_effect':'bi_effect','question_bbi_2016wave4_basicincome_argumentsfor':'bi_argsfor','question_bbi_2016wave4_basicincome_argumentsagainst':'bi_argsagsagainst'})
jobs_db.columns

Index(['person_id', 'country_code', 'rural', 'uuid.1', 'ed_level', 'job_type',
       'normalized_job_code', 'uuid.2', 'age', 'gender', 'dem_has_children',
       'age_group', 'uuid.3', 'bi_awareness', 'bi_vote', 'bi_effect',
       'bi_argsfor', 'bi_argsagsagainst'],
      dtype='object')

In [9]:
#Drop unnecessary columns
drop_cols=[i for i in list(jobs_db.columns) if i.startswith('uuid')]
jobs_db=jobs_db.drop(columns=drop_cols)
jobs_db.columns

Index(['person_id', 'country_code', 'rural', 'ed_level', 'job_type',
       'normalized_job_code', 'age', 'gender', 'dem_has_children', 'age_group',
       'bi_awareness', 'bi_vote', 'bi_effect', 'bi_argsfor',
       'bi_argsagsagainst'],
      dtype='object')

In [10]:
jobs_db=jobs_db.rename(columns={'normalized_job_code':'uuid'})
jobs_db['uuid']
jobs_db

,person_id,country_code,rural,ed_level,job_type,uuid,...,age_group,bi_awareness,bi_vote,bi_effect,bi_argsfor,bi_argsagsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,countryside,no,no,NaN,...,40_65,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,...,40_65,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,city,NaN,no,NaN,...,26_39,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,Country,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,...,40_65,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,city,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,...,40_65,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...
5,28583060-dbe2-0133-ff9c-0a81e8b09a82,AT,countryside,high,yes,27af8700f5577cec835acee2cb90a2ff,...,26_39,I understand it fully,I would vote against it,A basic income would not affect my work choices,It encourages financial independence and self-...,It is impossible to finance | It might encoura...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9643,7d3c5860-d8e2-0133-b635-0a81e8b09a82,SK,city,medium,no,NaN,...,14_25,I have never heard of it,I would probably vote for it,A basic income would not affect my work choices,"It increases solidarity, because it is funded ...",It might encourage people to stop working | It...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,SK,urban,high,yes,847165cfda6b1dc82ae22b967da8af2f,...,26_39,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance
9645,39f989f0-db52-0133-8482-0a81e8b09a82,SK,urban,high,yes,a4d5b8b38f9513825d0d94a981ebe962,...,40_65,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...


In [11]:
filtered=jobs_db.uuid.notnull()
jobs_db=jobs_db[filtered]

In [12]:
api_data['uuid']
api_data.size

624

In [13]:
df=pd.merge(jobs_db,api_data, left_on='uuid',right_on='uuid')
df

,person_id,country_code,rural,ed_level,job_type,uuid,...,bi_effect,bi_argsfor,bi_argsagsagainst,normalized_job_title,parent_uuid,suggestion
0,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,...,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,automatic data processing planner,0148f61d4227497728ce33490843d056,Automatic Data Processing Planner
1,cf24ed60-da3f-0133-0034-0a81e8b09a82,AT,Country,high,yes,861a9b9151e11362eb3c77ca914172d0,...,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,None of the above,automatic data processing planner,0148f61d4227497728ce33490843d056,Automatic Data Processing Planner
2,9ffc7390-dba5-0133-259b-0a81e8b09a82,BE,Country,high,yes,861a9b9151e11362eb3c77ca914172d0,...,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,It increases dependence on the state,automatic data processing planner,0148f61d4227497728ce33490843d056,Automatic Data Processing Planner
3,a6361230-da58-0133-cd5a-0a81e8b09a82,BE,urban,medium,yes,861a9b9151e11362eb3c77ca914172d0,...,A basic income would not affect my work choices,None of the above,Only the people who need it most should get so...,automatic data processing planner,0148f61d4227497728ce33490843d056,Automatic Data Processing Planner
4,1e2b4750-d987-0133-3c7c-0a81e8b09a82,BG,urban,high,yes,861a9b9151e11362eb3c77ca914172d0,...,A basic income would not affect my work choices,It reduces anxiety about financing basic needs,It might encourage people to stop working,automatic data processing planner,0148f61d4227497728ce33490843d056,Automatic Data Processing Planner
5,c878db70-d9a7-0133-5c31-0a81e8b09a82,CZ,Non-Rural,medium,yes,861a9b9151e11362eb3c77ca914172d0,...,‰Û_ work less,It reduces anxiety about financing basic needs...,It might encourage people to stop working | On...,automatic data processing planner,0148f61d4227497728ce33490843d056,Automatic Data Processing Planner
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5696,94332cb0-d9bc-0133-b92b-0a81e8b09a82,IT,city,low,yes,4cee16550636e292b8d136486fce943b,...,‰Û_ gain additional skills,It creates more equality of opportunity | It e...,Only the people who need it most should get so...,data capture clerk,bf88358c56bb6cbe7eabec38ae333d19,Data Capture Clerk
5697,1cb1aac0-d94c-0133-8baa-0a81e8b09a82,IT,Country,high,yes,4cee16550636e292b8d136486fce943b,...,‰Û_ look for a different job,It reduces anxiety about financing basic needs...,Only the people who need it most should get so...,data capture clerk,bf88358c56bb6cbe7eabec38ae333d19,Data Capture Clerk
5698,c8c33390-da69-0133-063a-0a81e8b09a82,PL,city,medium,yes,4cee16550636e292b8d136486fce943b,...,‰Û_ look for a different job,It reduces anxiety about financing basic needs,It might encourage people to stop working | It...,data capture clerk,bf88358c56bb6cbe7eabec38ae333d19,Data Capture Clerk


In [14]:
df=pd.merge(df, country_codes, left_on='country_code', right_on='country_code', how='inner')

In [15]:
df = df.replace(
        {'countryside': 'rural', 'city': 'Non-rural', 'urban': 'Non-rural', 'Country': 'rural', 'Rural': 'rural',
         'Non-Rural': 'Non-rural'})

In [16]:
df

,person_id,country_code,rural,ed_level,job_type,uuid,...,bi_argsfor,bi_argsagsagainst,normalized_job_title,parent_uuid,suggestion,country
0,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,Non-rural,high,yes,861a9b9151e11362eb3c77ca914172d0,...,It increases appreciation for household work a...,It might encourage people to stop working,automatic data processing planner,0148f61d4227497728ce33490843d056,Automatic Data Processing Planner,Austria
1,cf24ed60-da3f-0133-0034-0a81e8b09a82,AT,rural,high,yes,861a9b9151e11362eb3c77ca914172d0,...,It reduces anxiety about financing basic needs...,None of the above,automatic data processing planner,0148f61d4227497728ce33490843d056,Automatic Data Processing Planner,Austria
2,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,rural,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,...,It reduces anxiety about financing basic needs,None of the above,data coordinator,0b9dd32a367f4562ec77b993053d1910,Data Coordinator,Austria
3,ca3f65f0-d92b-0133-3959-0a81e8b09a82,AT,rural,medium,yes,049a3f3a2b5f85cb2971ba77ad66e10c,...,It reduces anxiety about financing basic needs...,Foreigners might come to my country and take a...,data coordinator,0b9dd32a367f4562ec77b993053d1910,Data Coordinator,Austria
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,Non-rural,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,...,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...,database developer,b90ca4df5690002377a7b0f1f3d40781,Database Developer,Austria
5,28583060-dbe2-0133-ff9c-0a81e8b09a82,AT,rural,high,yes,27af8700f5577cec835acee2cb90a2ff,...,It encourages financial independence and self-...,It is impossible to finance | It might encoura...,data entry specialist,bf88358c56bb6cbe7eabec38ae333d19,Data Entry Specialist,Austria
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4920,4ec02860-dc9b-0133-be14-0a81e8b09a82,CY,Non-rural,medium,yes,086f0063610283d1c23c44bca4ef92c0,...,It reduces anxiety about financing basic needs,Foreigners might come to my country and take a...,database administrator dba,b90ca4df5690002377a7b0f1f3d40781,Database Administrator (DBA),Cyprus
4921,6e8c05a0-d98b-0133-a515-0a81e8b09a82,MT,Non-rural,no,yes,a12a16280e22220f9354333f6fe1b22a,...,None of the above,None of the above,geographic information systems data administra...,bab6b228ad976cd2624acbcfccb8ad9b,Geographic Information Systems Data Administra...,Malta
4922,46f91e60-d8cc-0133-611d-0a81e8b09a82,MT,Non-rural,high,yes,970eb38c7b3eb080466052ea994af189,...,It increases appreciation for household work a...,Foreigners might come to my country and take a...,internet database specialist,a5773315d55245d7a3271314a564c32f,Internet Database Specialist,Malta


In [17]:
country job title gender quantity percentage

SyntaxError: invalid syntax (<ipython-input-17-c86795057c6d>, line 1)

In [18]:
df=df.rename(columns={'normalized_job_title':'job_title'})
df.columns

Index(['person_id', 'country_code', 'rural', 'ed_level', 'job_type', 'uuid',
       'age', 'gender', 'dem_has_children', 'age_group', 'bi_awareness',
       'bi_vote', 'bi_effect', 'bi_argsfor', 'bi_argsagsagainst', 'job_title',
       'parent_uuid', 'suggestion', 'country'],
      dtype='object')

In [19]:

df['percentage']=

SyntaxError: invalid syntax (<ipython-input-19-702b652945bc>, line 1)

In [20]:
t=df.groupby(['country','job_title','rural'])['person_id'].count()
t

country  job_title                                                                         rural    
Austria  automatic data processing customer liaison adp customer liaison                   Non-rural    1
                                                                                           rural        1
         automatic data processing planner                                                 Non-rural    1
                                                                                           rural        1
         automatic data processing systems security specialist adp systems security        Non-rural    1
                                                                                           rural        1
                                                                                                       ..
Sweden   geographic information systems database administrator gis database administrator  rural        1
         head of information systems and data proce

In [21]:
df['quantity'] = 1
df['quantity'] = df.groupby(['country', 'job_title', 'rural']).person_id.transform('count').reset_index()

In [22]:
datasubset=df[['country', 'job_title', 'rural','person_id']]
datasubset['quantity'] = 1
datasubset= datasubset.groupby(['country', 'job_title', 'rural'])['quantity'].count().reset_index()    

,country,job_title,rural,quantity,percentage,percentage_by_country
0,Austria,automatic data processing customer liaison adp...,Non-rural,1,0.047214,1.639344
1,Austria,automatic data processing customer liaison adp...,rural,1,0.047214,1.639344
2,Austria,automatic data processing planner,Non-rural,1,0.047214,1.639344
3,Austria,automatic data processing planner,rural,1,0.047214,1.639344
4,Austria,automatic data processing systems security spe...,Non-rural,1,0.047214,1.639344
5,Austria,automatic data processing systems security spe...,rural,1,0.047214,1.639344
...,...,...,...,...,...,...
2112,Sweden,geographic information systems database admini...,rural,1,0.047214,1.369863
2113,Sweden,head of information systems and data processing,Non-rural,1,0.047214,1.369863
2114,Sweden,naval tactical data systemcombat information c...,Non-rural,1,0.047214,1.369863


{'Austria': 61, 'Belgium': 82, 'Bulgaria': 66, 'Croatia': 39, 'Cyprus': 8, 'Czechia': 96, 'Denmark': 56, 'Estonia': 10, 'Finland': 58, 'France': 204, 'Germany': 224, 'Hungary': 96, 'Ireland': 41, 'Italy': 203, 'Latvia': 22, 'Lithuania': 25, 'Luxembourg': 8, 'Malta': 5, 'Netherlands': 104, 'Poland': 174, 'Portugal': 89, 'Romania': 127, 'Slovakia': 48, 'Slovenia': 17, 'Spain': 182, 'Sweden': 73}


In [75]:

f

,country,quantity_data_jobs_by_country
0,Austria,61
1,Belgium,82
2,Bulgaria,66
3,Croatia,39
4,Cyprus,8
5,Czechia,96
...,...,...
20,Portugal,89
21,Romania,127
22,Slovakia,48


,country,job_title,rural,quantity,percentage,quantity_data_jobs_by_country,percentage_by_country
0,Austria,automatic data processing customer liaison adp...,Non-rural,1,0.047214,61,1.639344
1,Austria,automatic data processing customer liaison adp...,rural,1,0.047214,61,1.639344
2,Austria,automatic data processing planner,Non-rural,1,0.047214,61,1.639344
3,Austria,automatic data processing planner,rural,1,0.047214,61,1.639344
4,Austria,automatic data processing systems security spe...,Non-rural,1,0.047214,61,1.639344
5,Austria,automatic data processing systems security spe...,rural,1,0.047214,61,1.639344
...,...,...,...,...,...,...,...
2112,Sweden,geographic information systems database admini...,rural,1,0.047214,73,1.369863
2113,Sweden,head of information systems and data processing,Non-rural,1,0.047214,73,1.369863
2114,Sweden,naval tactical data systemcombat information c...,Non-rural,1,0.047214,73,1.369863


In [27]:
table_1=df.groupby(['country','job_title','rural'])['quantity'].count()

In [31]:
table_1

country  job_title                                                                         rural    
Austria  automatic data processing customer liaison adp customer liaison                   Non-rural    1
                                                                                           rural        1
         automatic data processing planner                                                 Non-rural    1
                                                                                           rural        1
         automatic data processing systems security specialist adp systems security        Non-rural    1
                                                                                           rural        1
                                                                                                       ..
Sweden   geographic information systems database administrator gis database administrator  rural        1
         head of information systems and data proce

In [ ]:
df.columns

In [33]:
#CLEANING
df.isnull().sum()

person_id                 0
country_code              0
rural                     0
ed_level                256
job_type                  0
uuid                      0
                       ... 
bi_effect                 0
bi_argsfor                0
bi_argsagsagainst         0
normalized_job_title      0
parent_uuid               0
suggestion                0
Length: 18, dtype: int64

In [28]:
df.groupby('suggestion').agg('ed_level').count()

suggestion
Analytical Data Miner                                                           10
Automatic Data Processing Customer Liaison (ADP Customer Liaison)               88
Automatic Data Processing Planner                                               78
Automatic Data Processing Planner (ADP Planner)                                 77
Automatic Data Processing Systems Security Specialist (ADP Systems Security)    85
Business Database Analyst                                                       12
                                                                                ..
SCADA Technician (Supervisory Control and Data Acquisition Technician)          13
SQL Database Administrator                                                      65
Scientific Database Curator                                                      8
Survey Data Technician                                                          11
Voice and Data Technician                                                   

In [21]:
filtro=df['ed_level']=='no'
df[filtro]['suggestion'].unique()

array(['Automatic Data Processing Planner', 'Data Coordinator',
       'Database Developer', 'Data Entry Specialist',
       'Database Architect',
       'Geographic Information Systems Database Administrator (GIS Database Administrator)',
       'Crime Data Specialist', 'Data Communications Software Consultant',
       'Data Security Analyst', 'Database Marketing Analyst',
       'Computer or Data Processing Systems Consultant',
       'Database Development and Administration Project Manager',
       'Naval Tactical Data System--Combat Information Center Watch Officer, General',
       'Data Coder Operator',
       'Automatic Data Processing Customer Liaison (ADP Customer Liaison)',
       'Data Entry Representative', 'Data Processing Auditor',
       'Maintenance Data Analyst',
       'Automatic Data Processing Systems Security Specialist (ADP Systems Security)',
       'Database Engineer', 'Data Processing Systems Project Planner',
       'Database Administration Associate', 'Data W

In [22]:
df.groupby(['ed_level']).size()

ed_level
high      2450
low        777
medium    2111
no         108
dtype: int64

In [21]:
df[filter_c].groupby(['ed_level']).size()

ed_level
high      138
low        77
medium    146
no         15
dtype: int64

In [17]:
df=df.replace({'countryside':'rural', 'city':'Non-rural','urban':'Non-rural','Country':'rural','Rural':'rural','Non-Rural':'Non-rural'})

In [43]:
df['ed_level'].unique()

array(['high', 'medium', 'low', nan, 'no'], dtype=object)